In [ ]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_report
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ""

outdir = setup("20220708_vacc_projections.ipynb")

In [ ]:
model_args = {
    'params_defs': json.load(open('covid_model/analysis/20220629_july_gov_briefing/scenario_params.json')),
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20220629_july_gov_briefing/vacc_proj_params.json',
    'regions': ['co'],
    'mobility_mode': None,
    'start_date': '2020-01-24',
    'end_date': '2022-09-15',
    'max_step_size': 1.0,
    'ode_method': 'RK45',
    'hosp_reporting_frac': {"2020-01-01": 1, "2022-03-01": 0.8, "2022-03-15": 0.66, "2022-03-30": 0.5}
}
# create initial solution so we don't have to fit the whole window
model = CovidModel(**model_args)

In [ ]:
model.vacc_proj_params = 'covid_model/analysis/20220629_july_gov_briefing/vacc_proj_params.json'
model.update_data()

In [ ]:
df_actual = model.actual_vacc_df.stack(dropna=False).reset_index().rename(columns={'level_3': 'shot', 0: 'count'}).assign(type='actual')
df_proj = model.proj_vacc_df.stack(dropna=False).reset_index().rename(columns={'level_3': 'shot', 0: 'count'}).assign(type='projection')
df = pd.concat([df_actual, df_proj])
df.head()

In [ ]:
t1 = dt.datetime.strptime('2022-03-01', '%Y-%m-%d').date()
t2 = model.end_date

In [ ]:
filt = (t1 <= df.date) & (df.date <= t2)
rel = sns.relplot(data=df.loc[filt,:], kind='line', 
                  x='date', y='count', row='age', hue='shot', style='type', 
                  height=3, aspect=5, facet_kws={'sharey': False})